In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [63]:
def fork_archive(repo_name, repo_path):
    import pandas as pd
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT forked_from, repo_archived from repo
            WHERE repo_name = {repo_name}
            AND repo_path = {repo_path}
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    forked = repo_df.forked_from[0]
    archived = repo_df.repo_archived[0]
    
    is_archived = 'ERROR'
    
    if forked != 'Parent not available':
        is_forked = True
    else:
        is_forked = False
    
    if archived == 1:
       is_archived = True
    elif archived == 0:
       is_archived = False

    print(str(is_forked), str(is_archived))
    return is_forked, is_archived
    
#repo_name = "'turbine'"
#repo_path = "'github.com/concourse/'"
#repo_name = "'rabbithub'"
#repo_path = "'github.com/rabbitmq/'"
repo_name = "'cloudfoundry-sandbox'"
repo_path = "'github.com/Spring-projects/'"
is_fork, is_archived = fork_archive(repo_name, repo_path)

False False


In [14]:
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT repo_path, repo_name, forked_from from repo
            where forked_from NOT LIKE '%%Parent%%'
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    repo_df

,repo_path,repo_name,forked_from
0,github.com/carbonblack/,graphite-web,graphite-project/graphite-web
1,github.com/Pivotal-Data-Engineering/,cf-SpringBootTrader,pivotal-bank/cf-SpringBootTrader
2,github.com/Pivotal-DataFabric/,concourse-summary,dgodd/concourse-summary
3,github.com/cfmobile/,example-tile-docs-resources,pivotal-cf-experimental/example-tile-docs-reso...
4,github.com/cfmobile/,spring-amqp,spring-projects/spring-amqp
...,...,...,...
613,github.com/Pivotal-Field-Engineering/,sleuth-documentation-apps,spring-cloud-samples/sleuth-documentation-apps
614,github.com/Pivotal-Field-Engineering/,pse-training-cf-hw-module,vmwarepivotallabs/pse-training-cf-hw-module
615,github.com/Pivotal-Field-Engineering/,cf-cli-resource,nulldriver/cf-cli-resource
616,github.com/pivotal-cf-experimental/,fog,fog/fog


In [15]:
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT repo_path, repo_name, repo_archived from repo
            where repo_archived = 1
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    repo_df

,repo_path,repo_name,repo_archived
0,github.com/pivotal-cf/,react-starter,1
1,github.com/concourse/,turbine,1
2,github.com/concourse/,glider,1
3,github.com/concourse/,hangar,1
4,github.com/concourse/,blackbox,1
...,...,...,...
139,github.com/bitnami/,dockerode,1
140,github.com/bitnami/,minideb-extras,1
141,github.com/SteeltoeOSS/,Logging,1
142,github.com/SteeltoeOSS/,Common,1


In [11]:
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT forked_from, repo_archived from repo
            where forked_from NOT LIKE '%%Parent%%'
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    repo_df

,forked_from,repo_archived
0,graphite-project/graphite-web,0
1,pivotal-bank/cf-SpringBootTrader,0
2,dgodd/concourse-summary,0
3,pivotal-cf-experimental/example-tile-docs-reso...,0
4,spring-projects/spring-amqp,0
...,...,...
613,spring-cloud-samples/sleuth-documentation-apps,0
614,vmwarepivotallabs/pse-training-cf-hw-module,0
615,nulldriver/cf-cli-resource,0
616,fog/fog,0
